In [1]:
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints._base import Endpoint
from nba_api.stats.library.http import NBAStatsHTTP
from nba_api.stats.endpoints import playerdashboardbyteamperformance
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.endpoints import teamdashboardbyteamperformance
from nba_api.stats.endpoints import playerdashboardbylastngames
from nba_api.stats.endpoints import playerdashboardbyyearoveryear
from nba_api.stats.endpoints import teamdashboardbylastngames
from nba_api.stats.endpoints import teamplayerdashboard
from nba_api.stats.static import teams
from nba_api.stats.static import players
import pandas as pd
import inspect
import numpy as np
import csv
import sys

headers = { 'Host': 'stats.nba.com', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:61.0) Gecko/20100101 Firefox/61.0', 'Accept': 'application/json, text/plain, /', 'Accept-Language': 'en-US,en;q=0.5', 'Referer': 'https://stats.nba.com/', 'Accept-Encoding': 'gzip, deflate, br', 'Connection': 'keep-alive', }

In [18]:
#Player Input

player1_input = input('Enter player 1: ')
print(player1_input)

player1 = players.find_players_by_full_name(player1_input)[0]['id']
player1_name = players.find_players_by_full_name(player1_input)[0]['full_name']

player_info = playerdashboardbylastngames.PlayerDashboardByLastNGames(player_id=player1, 
                                                                      season='2018-19', headers=headers)

# player_headers = player_info.by_year_player_dashboard.get_dict().get('headers')
# player_data = player_info.overall_player_dashboard.get_dict().get('data')[0]

# #last 20 games
player_headers = player_info.last20_player_dashboard.get_dict().get('headers')
player_data = player_info.last20_player_dashboard.get_dict().get('data')[0]

print(player_headers[5], player_data[5], 
	player_headers[2], player_data[2], 
	player_headers[26], player_data[26],
	player_headers[27], player_data[27])

Enter player 1: lebron james
lebron james
W_PCT 0.35 GP 20 PTS 553 PLUS_MINUS 19


In [19]:
#Team Input

team1_input = input('Enter team 1: ')
print(team1_input)
team1 = teams.find_teams_by_full_name(team1_input)[0]['id']

teamdashboard = teamdashboardbylastngames.TeamDashboardByLastNGames(team_id=team1, season='2018-19', last_n_games=1, 
                                                                   headers=headers)
teamdashboard_adv = teamdashboardbylastngames.TeamDashboardByLastNGames(team_id=team1, season='2018-19', 
                                                                    measure_type_detailed_defense='Advanced', 
                                                                        last_n_games=1, headers=headers)

##last n games
team_headers_basic = teamdashboard.overall_team_dashboard.get_dict().get('headers')
team_data_basic = teamdashboard.overall_team_dashboard.get_dict().get('data')[0]
team_headers = teamdashboard_adv.overall_team_dashboard.get_dict().get('headers')
team_data = teamdashboard_adv.overall_team_dashboard.get_dict().get('data')[0]

##last 20 games
# team_headers_basic = teamdashboard.last20_team_dashboard.get_dict().get('headers')
# team_data_basic = teamdashboard.last20_team_dashboard.get_dict().get('data')[0]
# team_headers = teamdashboard_adv.last20_team_dashboard.get_dict().get('headers')
# team_data = teamdashboard_adv.last20_team_dashboard.get_dict().get('data')[0]

print(team1, team_headers, team_data)
# print(team1, team_headers[23], team_data[23], team_headers[8], team_data[8], team_headers[10], team_data[10], 
#       team_headers[2], team_data[2], team_headers[3], team_data[3], 
#       team_headers[4], team_data[4], team_headers[21], team_data[21], team_headers[6], team_data[6])

Enter team 1: lakers
lakers
1610612747 ['GROUP_SET', 'GROUP_VALUE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING', 'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TO', 'AST_RATIO', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'POSS', 'PIE', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'OFF_RATING_RANK', 'DEF_RATING_RANK', 'NET_RATING_RANK', 'AST_PCT_RANK', 'AST_TO_RANK', 'AST_RATIO_RANK', 'OREB_PCT_RANK', 'DREB_PCT_RANK', 'REB_PCT_RANK', 'TM_TOV_PCT_RANK', 'EFG_PCT_RANK', 'TS_PCT_RANK', 'PACE_RANK', 'PIE_RANK', 'CFID', 'CFPARAMS'] ['Overall', '2018-19', 1, 0, 1, 0.0, 48.0, 99.5, 102.0, 101.3, 105.1, -1.8, -3.0, 0.725, 1.71, 19.5, 0.393, 0.704, 0.545, 0.172, 0.479, 0.498, 102.1, 99.0, 82.5, 99, 0.486, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 163, '2018-19']


In [20]:
roster1 = teamplayerdashboard.TeamPlayerDashboard(team_id=team1, last_n_games=1, 
                                                  season='2018-19', headers=headers).get_dict()#.PlayersSeasonTotals(team_id=team1, last_n_games=20)
team1_roster_last20 = []
team1_roster_lastn = []
#print(roster1)['resultSets'][1])#['rowSet'])
for n in roster1['resultSets'][1]['rowSet']:
    print(n[2], 'GP:', n[3], 'id:', n[1], 'Min', n[7])
    team1_roster_lastn.append(n[2])
    
print(team1_roster_lastn)
#(PER * MP)/n_players

Alex Caruso GP: 1 id: 1627936 Min 40.56666666666667
Isaac Bonga GP: 1 id: 1629067 Min 9.08
JaVale McGee GP: 1 id: 201580 Min 19.416666666666668
Jemerrio Jones GP: 1 id: 1629203 Min 46.483333333333334
Johnathan Williams GP: 1 id: 1629140 Min 28.583333333333332
Kentavious Caldwell-Pope GP: 1 id: 203484 Min 43.95
Mike Muscala GP: 1 id: 203488 Min 35.403333333333336
Moritz Wagner GP: 1 id: 1629021 Min 16.516666666666666
['Alex Caruso', 'Isaac Bonga', 'JaVale McGee', 'Jemerrio Jones', 'Johnathan Williams', 'Kentavious Caldwell-Pope', 'Mike Muscala', 'Moritz Wagner']


In [114]:
def off_per_calculator(player, team):
    
    agg_PER = 0 
    agg_adj_PER = 0
    num_players = 0
    PER_MP = 0
    agg_min = 0
    
    for n in player:
        p_id = players.find_players_by_full_name(n)[0]['id']
        p_info = playerdashboardbylastngames.PlayerDashboardByLastNGames(player_id=p_id, season='2018-19',
                                                                        last_n_games=1, headers=headers)
        
        ##last 20 games        
#         player_headers = p_info.last20_player_dashboard.get_dict().get('headers')
#         player_data = p_info.last20_player_dashboard.get_dict().get('data')[0]
        
        player_headers = p_info.overall_player_dashboard.get_dict().get('headers')
        player_data = p_info.overall_player_dashboard.get_dict().get('data')[0]

 
    #####PER player stats####
        MP = player_data[6]
        ThrP = player_data[10] 
        AST = player_data[19]
        TOV = player_data[20]
        FGA = player_data[8]
        FG = player_data[7] + player_data[10] 
        FTA = player_data[14]
        FT = player_data[13]
        TRB = player_data[18]
        ORB = player_data[16]
        STL = player_data[21]
        BLK = player_data[22]
        PF = player_data[24]

    ###PER league stats (2018-19)###
        lg_AST = 24.6
        lg_FG = 41.1
        lg_PTS = 111.2
        lg_FGA = 89.2
        lg_FT = 17.7
        lg_FTA = 23.1
        lg_TRB = 45.2
        lg_ORB = 10.3
        lg_TOV = 14.1
        lg_PF = 20.9
        lg_pace = 100
        
    ###PER team stats###
        team_AST = team_data_basic[19]
        team_FG = team_data_basic[7]+team_data_basic[10]
        team_pace = team_data[23] 
       
        factor = (2/3) - (0.5*(lg_AST/lg_FG)) / (2*(lg_FG/lg_FT)) 
        VOP = lg_PTS / (lg_FGA-lg_ORB+lg_TOV+0.44*lg_FTA) 
        DRB_perc = (lg_TRB -lg_ORB) / lg_TRB

    ####PER Calculation####                    
        PER_calc_player = (1 / MP) * ( ThrP + (2/3) * AST 
                                      + (2 - factor * (team_AST / team_FG)) * FG 
                                      + (FT *0.5 * (1 + (1 - (team_AST / team_FG)) + (2/3) * (team_AST / team_FG)))
                                      - VOP * TOV - VOP * DRB_perc * (FGA - FG)
                                      - VOP * 0.44 * (0.44 + (0.56 * DRB_perc)) * (FTA - FT)
                                      + VOP * (1 - DRB_perc) * (TRB - ORB) + VOP * DRB_perc * ORB 
                                      + VOP * STL + VOP * DRB_perc * BLK - PF 
                                      * ((lg_FT / lg_PF) - 0.44 * (lg_FTA / lg_PF) * VOP) )

        adj_PER_calc_player = PER_calc_player * (lg_pace /team_pace)
        
        agg_PER += PER_calc_player 
        agg_adj_PER += adj_PER_calc_player
        num_players +=1
        PER_MP += (adj_PER_calc_player * MP)
        PER_MP_n = PER_MP/num_players
        
        print(n, 'Min', MP, 'adj_PER_calc_player', adj_PER_calc_player, 'PER*MP_Player', (adj_PER_calc_player*MP))
        
    print('Agg_PER * MP', PER_MP)
    
    print('PERxMP/n', PER_MP_n, 'Total PER', agg_PER, 'Total adj_PER', agg_adj_PER)
    print('team ast', team_AST, 'team FG', team_FG, 'team_pace', team_pace)
#    print('league stats:', factor, VOP, DRB_perc)

In [115]:
off_per_calculator(team1_roster_lastn, team1)

Danilo Gallinari Min 24.8 adj_PER_calc_player 0.426271056923702 PER*MP_Player 10.57152221170781
Garrett Temple Min 21.583333333333332 adj_PER_calc_player 0.3514380730823768 PER*MP_Player 7.585205077361299
Ivica Zubac Min 29.783333333333335 adj_PER_calc_player 0.6585686980390959 PER*MP_Player 19.61437105659774
JaMychal Green Min 25.333333333333332 adj_PER_calc_player 0.5061284203324381 PER*MP_Player 12.821919981755098
Jerome Robinson Min 16.45 adj_PER_calc_player 0.22087037238040508 PER*MP_Player 3.6333176256576634
Landry Shamet Min 29.416666666666668 adj_PER_calc_player 0.5033830669166712 PER*MP_Player 14.807851885132079
Lou Williams Min 23.316666666666666 adj_PER_calc_player 0.3062138270859272 PER*MP_Player 7.139885734886869
Montrezl Harrell Min 20.816666666666666 adj_PER_calc_player 0.9940717283930562 PER*MP_Player 20.693259812715453
Patrick Beverley Min 21.316666666666666 adj_PER_calc_player 1.013862675943175 PER*MP_Player 21.612172708855347
Shai Gilgeous-Alexander Min 24.1 adj_PER_

In [21]:
#Alternate Offensive Rating

def off_per_calc_alt(player, team):
    
    agg_off_rating = 0
    agg_min = 0

    player_info_off = leaguedashplayerstats.LeagueDashPlayerStats(last_n_games=5, season='2018-19', 
                                                                  measure_type_detailed_defense='Advanced', 
                                                                  headers=headers)

    player_off_headers = player_info_off.league_dash_player_stats.get_dict().get('headers')
    player_off_data = player_info_off.league_dash_player_stats.get_dict().get('data')

#     print('headers:', player_off_headers)
    #print('data:', player_def_data)

    for p in team1_roster_lastn:
        for n in player_off_data:
            if p in n:
                print(n[1], '\n', 'E_Off_Rating', n[10], 'Min', n[9])
                agg_off_rating += (n[10]*n[9])
                agg_min += n[9]

    avg_off_rating = agg_off_rating/len(team1_roster_lastn)

    print('agg_off_rating', agg_off_rating, 'avg_off_rating', avg_off_rating, 'agg_min', agg_min, 
          'agg_off_rat_min', agg_off_rating/agg_min) 

In [107]:
#Individual Player Defense Stats

def def_per_calc(team_roster):
    agg_def_rating = 0
    agg_min = 0

    # player_info_def = leaguedashplayerstats.LeagueDashPlayerStats(last_n_games=1, season='2018-19', 
    #                                                                   measure_type_detailed_defense='Defense')
    player_info_def = leaguedashplayerstats.LeagueDashPlayerStats(opponent_team_id=, month=4, season='2018-19', 
                                                                      measure_type_detailed_defense='Defense',
                                                                 headers=headers)



    player_def_headers = player_info_def.league_dash_player_stats.get_dict().get('headers')
    player_def_data = player_info_def.league_dash_player_stats.get_dict().get('data')

    print('headers:', player_def_headers)
        #print('data:', player_def_data)

    for p in team1_roster_lastn:
        for n in player_def_data:
            if p in n:
                print(n[1], '\n', 'Def Rating', n[10], 'Min', n[9])
                agg_def_rating += (n[10]*n[9])
                agg_min += n[9]


    avg_def_rating = agg_def_rating/len(team1_roster)
    avg_def_rating_min = agg_def_rating/agg_min

    print('agg_def_rating', agg_def_rating, 'avg_def_rating', avg_def_rating, 'agg_min', agg_min) 
    print('avg_def_rating_min', avg_def_rating_min)

In [4]:
def off_per_calculator_pandas(player, team):
    
    agg_PER = 0 
    agg_adj_PER = 0
    num_players = 0
    PER_MP = 0
    agg_min = 0

    
    for n in player:
#         p_id = players.find_players_by_full_name(n)[0]['id']
#         p_info = playerdashboardbylastngames.PlayerDashboardByLastNGames(player_id=p_id, season='2018-19',
#                                                                         last_n_games=1)
        
#         ##last 20 games        
# #         player_headers = p_info.last20_player_dashboard.get_dict().get('headers')
# #         player_data = p_info.last20_player_dashboard.get_dict().get('data')[0]
        
#         player_headers = p_info.overall_player_dashboard.get_dict().get('headers')
#         player_data = p_info.overall_player_dashboard.get_dict().get('data')[0]

 
    #####PER player stats####
        MP = team1_roster['MINUTES']
        ThrP = team1_roster['3PMa'] 
        AST = team1_roster['AST']
        TOV = team1_roster['TOV']
        FGA = team1_roster['FGA']
        FG = team1_roster['FGM'] 
        FTA = team1_roster['FTA']
        FT = team1_roster['FTM']
        TRB = team1_roster['REB']
        ORB = team1_roster['OREB']
        STL = team1_roster['STL']
        BLK = team1_roster['BLK']
        PF = team1_roster['PF']

    ###PER league stats (2018-19)###
        lg_AST = 24.6
        lg_FG = 41.1
        lg_PTS = 111.2
        lg_FGA = 89.2
        lg_FT = 17.7
        lg_FTA = 23.1
        lg_TRB = 45.2
        lg_ORB = 10.3
        lg_TOV = 14.1
        lg_PF = 20.9
        lg_pace = 100
        
    ###PER team stats###
        team_AST = 25.6
        team_FG = 42.6
        team_pace = 103.6
       
        factor = (2/3) - (0.5*(lg_AST/lg_FG)) / (2*(lg_FG/lg_FT)) 
        VOP = lg_PTS / (lg_FGA-lg_ORB+lg_TOV+0.44*lg_FTA) 
        DRB_perc = (lg_TRB -lg_ORB) / lg_TRB

    ####PER Calculation####                    
        PER_calc_player = (1 / MP) * ( ThrP + (2/3) * AST 
                                      + (2 - factor * (team_AST / team_FG)) * FG 
                                      + (FT *0.5 * (1 + (1 - (team_AST / team_FG)) + (2/3) * (team_AST / team_FG)))
                                      - VOP * TOV - VOP * DRB_perc * (FGA - FG)
                                      - VOP * 0.44 * (0.44 + (0.56 * DRB_perc)) * (FTA - FT)
                                      + VOP * (1 - DRB_perc) * (TRB - ORB) + VOP * DRB_perc * ORB 
                                      + VOP * STL + VOP * DRB_perc * BLK - PF 
                                      * ((lg_FT / lg_PF) - 0.44 * (lg_FTA / lg_PF) * VOP) )

        adj_PER_calc_player = PER_calc_player * (lg_pace /team_pace)
        
        agg_PER += PER_calc_player 
        agg_adj_PER += adj_PER_calc_player
        num_players +=1
        PER_MP = adj_PER_calc_player * MP
        PER_MP_n += PER_MP
        
        print(n, 'Min', MP, 'adj_PER_calc_player', adj_PER_calc_player, 'PER*MP_Player', (adj_PER_calc_player*MP))
        
    print('Agg_PER * MP_n', PER_MP)
    
#     print('PERxMP/n', PER_MP_n, 'Total PER', agg_PER, 'Total adj_PER', agg_adj_PER)
    print('team ast', team_AST, 'team FG', team_FG, 'team_pace', team_pace)
#    print('league stats:', factor, VOP, DRB_perc)

In [2]:
data = pd.read_csv("NBA Merged Sheets - Sheet2.csv") 
data_t = pd.read_csv("team_stats_basic_2018-19.csv")
df = pd.read_csv("team 2018-19.csv") 

teams = ['ATL', 'IND', 'MIL', 'ORL', 'PHI', 'SAS', 'POR', 'NOP', 'UTA', 'HOU', 'BOS', 
         'MEM', 'BKN', 'MIA', 'CHI', 'MIN', 'PHX', 'DET', 'NYK', 'LAL', 'CHA', 'TOR', 
         'WAS', 'SAC', 'LAC', 'OKC', 'CLE', 'DAL', 'DEN', 'GSW']


In [24]:
team1 = 'ATL'
team2 = []
date = []

score = []

df2 = df[['Team', 'Matchup', 'Date', 'OffRtg', 'DefRtg', 'ASTRatio', 'PACE']]
df2 = df[df['Team'] == team1]

opp = []

team2 = df2['Matchup']

for row in team2:
    opp.append(row.replace(team1+'vs.', '').replace(team1+'@', ''))

df2['Opp'] = opp

team1_off_avg = 0
team1_def_avg = 0

team2_off_avg = 0
team2_def_avg = 0


for index, row in df2.iterrows(): 
    
    team2 = row['Opp']
    date = row['Date']
    df3 = df[df['Team'] == team2]

    score = data.loc[(data['DATE']==date) & (data['TEAM']==team1)]
    score = score.iloc[0]
    score = (score["team_pts"], score['team_plus_minus'])
    print('===', row['Team'], row['OffRtg'], row['DefRtg'], 
          row['Date'], row['Opp'], score, "===") 

    
    count = 1
    while count <6:
#         print(df2.iloc[count], df2.iloc[count]['OffRtg'])
        team1_off_avg += df2.iloc[count]['OffRtg']
        team1_def_avg += df2.iloc[count]['DefRtg']
        print(df2.iloc[count]['Team'], team1_off_avg)
        
        team2_off_avg += df3.iloc[count]['OffRtg']
        team2_def_avg += df3.iloc[count]['DefRtg']
        print(df3.iloc[count]['Team'], team2_off_avg)
        count+=1
    
    team1_off_avg = team1_off_avg/5
    print("HIHIHIHI:", team1_off_avg)
#     team1_def_avg = team1_def_avg/5
    team2_off_avg = team2_off_avg/5
    print("Hihihihihi2:", team2_off_avg, '\n')
#     team2_def_avg = team2_def_avg/5

#     print('Team1:', team1, team1_off_avg, team1_def_avg, '\n', 
#           "Team2:", team2, team2_off_avg, team2_def_avg, '\n', 
#           "Score:", score )
    
    
    
# off_per_calculator_pandas(team1_roster_players, team1)
     
    
# # def_per_calc(team1_roster)

/Users/stan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


=== ATL 122.9 125.0 04/10/19 IND (134, -1.0) ===
ATL 99.1
IND 99.0
ATL 201.8
IND 198.0
ATL 328.0
IND 309.3
ATL 436.8
IND 422.6
ATL 551.1
IND 543.4
HIHIHIHI: 110.22
Hihihihihi2: 108.67999999999999 

=== ATL 99.1 106.5 04/07/19 MIL (107, -8.0) ===
ATL 209.32
MIL 215.18
ATL 312.02
MIL 339.48
ATL 438.21999999999997
MIL 452.78000000000003
ATL 547.02
MIL 568.6800000000001
ATL 661.3199999999999
MIL 684.08
HIHIHIHI: 132.26399999999998
Hihihihihi2: 136.816 

=== ATL 102.7 134.2 04/05/19 ORL (113, -36.0) ===
ATL 231.36399999999998
ORL 249.416
ATL 334.06399999999996
ORL 383.616
ATL 460.26399999999995
ORL 497.616
ATL 569.064
ORL 605.516
ATL 683.3639999999999
ORL 731.516
HIHIHIHI: 136.6728
Hihihihihi2: 146.3032 

=== ATL 126.2 118.4 04/03/19 PHI (130, 8.0) ===
ATL 235.7728
PHI 241.5032
ATL 338.4728
PHI 361.1032
ATL 464.6728
PHI 468.1032
ATL 573.4728
PHI 586.5032
ATL 687.7728
PHI 681.8032
HIHIHIHI: 137.55455999999998
Hihihihihi2: 136.36064 

=== ATL 108.8 113.6 04/02/19 SAS (111, -6.0) ===
ATL 236.6

ATL 688.8749999999999
TOR 724.1390787319488
HIHIHIHI: 137.77499999999998
Hihihihihi2: 144.82781574638975 

=== ATL 109.3 83.7 01/06/19 MIA (106, 24.0) ===
ATL 236.87499999999997
MIA 261.02781574638976
ATL 339.575
MIA 363.8278157463898
ATL 465.775
MIA 471.72781574638975
ATL 574.5749999999999
MIA 576.9278157463898
ATL 688.8749999999999
MIA 681.9278157463898
HIHIHIHI: 137.77499999999998
Hihihihihi2: 136.38556314927797 

=== ATL 99.1 127.4 01/04/19 MIL (112, -32.0) ===
ATL 236.87499999999997
MIL 242.88556314927797
ATL 339.575
MIL 367.185563149278
ATL 465.775
MIL 480.485563149278
ATL 574.5749999999999
MIL 596.385563149278
ATL 688.8749999999999
MIL 711.785563149278
HIHIHIHI: 137.77499999999998
Hihihihihi2: 142.3571126298556 

=== ATL 98.0 114.0 01/02/19 WAS (98, -16.0) ===
ATL 236.87499999999997
WAS 254.55711262985562
ATL 339.575
WAS 371.2571126298556
ATL 465.775
WAS 479.8571126298556
ATL 574.5749999999999
WAS 575.8571126298556
ATL 688.8749999999999
WAS 692.8571126298556
HIHIHIHI: 137.774999

In [60]:
#Theory: If second game of back-to-back is a road game, odds of winning decrease 7-8%, multiply ratings by 0.93.